

#Packages

In [ ]:
globals().clear()

In [ ]:
import os
import pandas as pd
!pip install catboost
!pip install fredapi
import numpy as np
from itertools import product
!pip install category_encoders

from sklearn.neural_network import MLPRegressor

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression

# --- target encoding ---
from category_encoders.target_encoder import TargetEncoder
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline


from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer

from sklearn.ensemble import (
    ExtraTreesRegressor, RandomForestRegressor,
    GradientBoostingRegressor, AdaBoostRegressor
)
from sklearn.tree import DecisionTreeRegressor
from scipy import sparse
from google.colab import drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data preparation

In [ ]:
from fredapi import Fred

API_KEY = "XXXXXXXXXXXXXXXXXXXXX"
fred = Fred(api_key=API_KEY)
def _suiv_month_id(series):

    s = series.astype(str).str[:6]                     # 'YYYYMM'
    dt = pd.to_datetime(s + "01", format="%Y%m%d")
    suiv= (dt.dt.to_period("M") + 1).dt.strftime("%Y%m")
    return suiv.astype(int)


FRED_SERIES = {
    # energy (daily)
    "Crude Oil (WTI)": "DCOILWTICO",        # WTI (USD/baril)
    "Brent Oil": "DCOILBRENTEU",            # Brent (USD/baril)
    "Natural Gas (Henry Hub)": "DHHNGSP",   # Gaz naturel spot US (USD/MMBtu)

    # metal (IMF, monthly)
    "Copper": "PCOPPUSDM",
    "Aluminum": "PALUMUSDM",
    "Nickel": "PNICKUSDM",
    "Zinc": "PZINCUSDM",
    "Tin": "PTINUSDM",
    "Lead": "PLEADUSDM",
    "Iron Ore": "PIORECRUSDM",

    # Agriculture (IMF, monthly)
    "Wheat": "PWHEAMTUSDM",
    "Corn": "PMAIZMTUSDM",
    "Soybeans": "PSOYBUSDM",
    "Rice (Thailand)": "PRICENPQUSDM",
    "Coffee (Arabica - Other Mild)": "PCOFFOTMUSDM",
    "Cocoa": "PCOCOUSDM",
    "Sugar No.11 (World)": "PSUGAISAUSDM",
    "Cotton": "PCOTTINDUSDM",
}
def fetch_fred_monthly(series_dict,start="2020-12-31", end="2025-09-30"):
    all_data = []
    for name, code in series_dict.items():
        # daily
        s = fred.get_series(code, observation_start=start, observation_end=end)
        s = s.to_frame(name=name)

        # end of month
        s = s.resample("M").last()

        all_data.append(s)

    # Concat
    df = pd.concat(all_data, axis=1)

    # Add month_id

    df["month_id"] = df.index.strftime("%Y%m")
    cols = ["month_id"] + [c for c in df.columns if c != "month_id"]
    df = df[cols].reset_index(drop=True)
    df["month_id_join"] = _suiv_month_id(df["month_id"])
    df = df.drop(columns=["month_id"])

    return df
matieres_premiere= fetch_fred_monthly(FRED_SERIES)


In [ ]:

USA_export_2022=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/USA_2022_export_Additional.csv")
USA_import_2022=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/USA_2022_import_Additional.csv")
USA_export_2021=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/USA_2021_export_Additional.csv")
USA_import_2021=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/USA_2021_import_Additional.csv")
USA_export_2023=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/USA_2023_exports_Additional.csv")
USA_import_2023=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/USA_2023_imports_Additional.csv")
USA_export_2024=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/USA_2024_exports_Additional.csv")
USA_import_2024=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/USA_2024_imports_Additional.csv")
USA_export_2025=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/USA_2025_export_Additional.csv")
USA_import_2025=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/USA_2025_import_Additional.csv")

china_export_2022=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/china_2022_export_Additional.csv")
china_import_2022=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/china_2022_import_Additional.csv")
china_export_2021=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/china_2021_export_Additional.csv")
china_import_2021=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/china_2021_import_Additional.csv")
china_export_2023=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/china_2023_exports_Additional.csv")
china_import_2023=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/china_2023_imports_Additional.csv")
china_export_2024=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/china_2024_exports_Additional.csv")
china_import_2024=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/china_2024_imports_Additional.csv")
china_import_2025=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/China_2025_import_Additional.csv")
china_export_2025=pd.read_csv("/content/drive/MyDrive/ai4trade_cf/pre_processed_final/China_2025_export_Additional.csv")

china_import_2025['trade_flow_name'] = 'Import'
china_export_2025['trade_flow_name'] = 'Export'
USA_all=pd.concat([USA_export_2021, USA_import_2021,USA_export_2022, USA_import_2022,USA_export_2023, USA_import_2023,USA_export_2024, USA_import_2024,USA_export_2025, USA_import_2025], ignore_index=True)
china_all=pd.concat([china_export_2021,china_import_2021,china_export_2022,china_import_2022,china_export_2023,china_import_2023,china_export_2024,china_import_2024,china_import_2025,china_export_2025], ignore_index=True)

USA_all['product_id_hs4'] = USA_all['product_id_hs4'].astype(str).str.zfill(4)
china_all['product_id_hs4'] = china_all['product_id_hs4'].astype(str).str.zfill(4)

USA_all= USA_all.drop_duplicates().reset_index(drop=True)
china_all= china_all.drop_duplicates().reset_index(drop=True)

USA_all   = USA_all.loc[~USA_all["country_id"].isin(["W00", "S19"])]
china_all = china_all.loc[~china_all["country_id"].isin(["W00", "S19"])]
# 1. sum of trade vakues
USA_sum = USA_all.groupby(["month_id", "country_name", "country_id","trade_flow_name"], as_index=False)["trade_value"].sum()

# 2. Top 30
top_USA_sum = (
    USA_sum.groupby(["month_id","trade_flow_name"])
          .apply(lambda x: x.nlargest(30, "trade_value"))
          .reset_index(drop=True)[["month_id", "country_name", "country_id","trade_flow_name"]]
)

# 3. Subset
USA= USA_all.merge(top_USA_sum, on=["month_id", "country_name", "country_id","trade_flow_name"], how="inner")

# 1. sum of trade vakues
china_sum =china_all.groupby(["month_id", "country_name", "country_id","trade_flow_name"], as_index=False)["trade_value"].sum()

# 2. Top 30
top_china_sum = (
    china_sum.groupby(["month_id","trade_flow_name"])
          .apply(lambda x: x.nlargest(30, "trade_value"))
          .reset_index(drop=True)[["month_id", "country_name", "country_id","trade_flow_name"]]
)

# 3. Subset
china= china_all.merge(top_china_sum, on=["month_id", "country_name", "country_id","trade_flow_name"], how="inner")
USA = USA.drop(['quantity','nb_product','country_name','product_name_hs4'], axis=1)
china = china.drop(['quantity','nb_product','country_name','product_name_hs4'], axis=1)


china=china.merge(matieres_premiere, left_on="month_id",right_on="month_id_join", how="left").drop(columns="month_id_join")
USA=USA.merge(matieres_premiere,  left_on="month_id",right_on="month_id_join", how="left").drop(columns="month_id_join")

# creation of  202510
USA_202510_index = USA[['country_id','trade_flow_name','product_id_hs4']].drop_duplicates()
USA_202510_index['month_id'] = 202510

USA = pd.concat([USA, USA_202510_index], ignore_index=True)


china_202510_index = china[['country_id','trade_flow_name','product_id_hs4']].drop_duplicates()
china_202510_index['month_id'] = 202510

china = pd.concat([china, china_202510_index], ignore_index=True)

# corrections
flow_map = {
    'Exports': 'Export',
    'Export': 'Export',
    'Imports': 'Import',
    'Import': 'Import'
}

# normalisation
USA['trade_flow_name'] = USA['trade_flow_name'].replace(flow_map)
USA['country_id'] = USA['country_id'].str.upper()



china['trade_flow_name'] = china['trade_flow_name'].replace(flow_map)
china['country_id'] = china['country_id'].str.upper()

USA= USA.drop_duplicates().reset_index(drop=True)
china= china.drop_duplicates().reset_index(drop=True)
del  USA_202510_index,china_202510_index,  USA_export_2022, USA_import_2022,USA_export_2021,USA_import_2021,USA_export_2023,USA_import_2023,USA_export_2024,USA_import_2024,USA_export_2025,USA_import_2025,china_export_2022,china_import_2022,china_export_2021,china_export_2023,china_import_2023,china_import_2024,top_china_sum,china_sum,top_USA_sum,china_export_2024,china_all,USA_sum,USA_all



# data quality checks

In [ ]:
china_duplicates_count = (china.groupby(['country_id','product_id_hs4','trade_flow_name','month_id'])
                           .size()
                           .reset_index(name='count')
                           .query('count > 1')
                        )

print(china_duplicates_count)

usa_duplicates_count = (USA.groupby(['country_id','product_id_hs4','trade_flow_name','month_id'])
                           .size()
                           .reset_index(name='count')
                           .query('count > 1')
                        )

print(usa_duplicates_count)


Empty DataFrame
Columns: [country_id, product_id_hs4, trade_flow_name, month_id, count]
Index: []
Empty DataFrame
Columns: [country_id, product_id_hs4, trade_flow_name, month_id, count]
Index: []


In [ ]:
print(china['month_id'].unique())

In [ ]:
print(USA['month_id'].unique())

#SABLE-12 — Seasonal Anchor with Boosted Lagged Exogènes

#Amélioration

In [ ]:
pip install xlsxwriter

# 🇺🇸 Full Mathematical Description of the Forecast Model

This document provides a **comprehensive mathematical explanation** of a forecasting model for **monthly trade values** by `(product × country × flow)`.

The model combines **naive baselines**, **seasonal and trend decomposition**, **lagged exogenous variables**, and a **robust Huber regression correction**, with **special handling for short or rare series**.

---

## 0. Notation

Let each series $s = (\text{product}, \text{country}, \text{flow})$ be observed monthly as:

$$
\{ (t_1, y_1), (t_2, y_2), ..., (t_T, y_T) \}, \quad y_t \ge 0
$$

We denote:
- $M_t \in \{1,\dots,12\}$ the **month** of $t$  
- $b_t$ the **base forecast**  
- $\hat y_t$ the **final prediction**  
- $x_t$ exogenous variables, possibly lagged

---

## 1. Base Forecast Candidates

For each series, several naive forecast candidates are computed:

1. **Last observed value**
$$
\text{last} = y_T
$$

2. **Same month last year**
$$
\text{seas12} = y_{T-12}, \quad \text{if available}
$$

3. **Moving averages**
$$
\text{ma3} = \frac{1}{3} \sum_{i=T-2}^{T} y_i, \quad
\text{ma6} = \frac{1}{6} \sum_{i=T-5}^{T} y_i
$$

4. **Drift extrapolation**
- Compute log-transformed slope over last 6 months:
$$
\beta = \frac{\sum_{i=T-5}^{T} (i-\bar i) (\log(1+y_i) - \overline{\log(1+y)} )}{\sum_{i=T-5}^{T} (i-\bar i)^2}
$$
- Extrapolated forecast:
$$
\text{drift_last} = y_T \cdot e^\beta
$$

5. **Seasonal-trend forecast** (see Section 2)

6. **Same-month median of last 3 years**
$$
\text{same_month_med3} = \text{median}\{y_t : M_t = M_T, t \in \text{last 3 years}\}
$$

**Base forecast** $b_{T+1}$ is the median of all valid candidates:

$$
b_{T+1} = \text{median}\{\text{last}, \text{seas12}, \text{ma3}, \text{ma6}, \text{drift_last}, \text{seasonal_ST}, \text{same_month_med3}\}
$$

---

## 2. Seasonal + Trend Decomposition

Assume a multiplicative model:

$$
y_t \approx g \cdot S_{M_t} \cdot \text{trend}_t
$$

### 2.1 Seasonal factor $S_m$

- Compute global scale $g$ as median of positive $y_t$ values:
$$
g = \text{median} \{ y_t : y_t > 0 \}
$$
- For each month $m$:
$$
S_m = \text{clip}\left( \frac{\text{median}\{y_t : M_t = m, y_t>0\}}{g}, 0.2, 5.0 \right)
$$

### 2.2 Deseasonalize and slope estimation

- Deseasonalized series:
$$
d_t = \frac{y_t}{S_{M_t}}
$$
- Slope on log-space (small window $w$):
$$
\beta = \frac{\sum_{i=T-w+1}^{T} (i-\bar i) (\log(1 + d_i) - \overline{\log(1+d)})}{\sum_{i=T-w+1}^{T} (i-\bar i)^2}
$$

### 2.3 Forecast next month

$$
\hat d_{T+1} = d_T \cdot e^\beta, \quad
\text{seasonal_ST} = \hat d_{T+1} \cdot S_{M_{T+1}}
$$

- Blend with same-month median and last value:
$$
\text{seasonal_ST} = \text{median}\{\text{seasonal_ST}, y_T, \text{same_month_med3}\}
$$

---

## 3. Exogenous Variable Selection

For candidate numeric exogenous variables $x_j$:

1. For each lag $L \in \{1,3,6\}$:
   - Form pairs $(x_{t-L}, y_t)$ over all series
2. Compute Spearman correlation $\rho_{x_j, L}$
3. Keep top `$k$` pairs $(x,L)$ with highest $|\rho_{x,L}|$

These features are used in the Huber regression.

---

## 4. Feature Construction

For each series and target month:

| Feature | Formula / Description |
|:--|:--|
| Month | $M_{T+1}$ |
| Binary month indicators | `is_jan`, `is_dec` |
| Encodings | Mean target per product/country/flow: $\text{enc\_prod} = \text{mean}(y)$ for that product |
| Seasonality | $S_{M_{T+1}}$ |
| Lags | $y_T$, $y_{T-1}$, $y_{T-2}$, $y_{T-3}$, $y_{T-6}$, $y_{T-12}$ |
| Exogenous | Selected top-$k$ lagged $x_j$ |
| Optional: log slope | $\beta$ from deseasonalized last $w$ points |

---

## 5. Robust Correction: Huber Regression

### 5.1 Target transformation

Define relative correction in log-space:

$$
z_t = \log(1 + y_t) - \log(1 + b_t)
$$

- $b_t$ is base forecast
- $z_t$ is the **residual correction**

### 5.2 Model training

- Stack all series and backtests
- Train **HuberRegressor** to predict $z_t$ from features $X_t$:

$$
\hat z_{T+1} = f_{\text{Huber}}(X_{T+1})
$$

Huber loss:
$$
L_\delta(r) =
\begin{cases}
\frac{1}{2} r^2 & |r| \le \delta \\
\delta (|r| - \frac{1}{2}\delta) & |r| > \delta
\end{cases}, \quad r = z_t - \hat z_t
$$

---

## 6. Final Forecast

- Log-space corrected forecast:
$$
\hat y_{\text{corr}} = \exp(\log(1+b_{T+1}) + \hat z_{T+1}) - 1
$$

- For rare series (mostly zeros):
$$
y_{\text{corr}} = \max(y_{\text{floor}}, \min(\hat y_{\text{corr}}, y_{T} \cdot g_{\text{cap}}))
$$

- Weighted blend with base forecast:
$$
\hat y_{T+1} = w \cdot y_{\text{corr}} + (1-w) \cdot b_{T+1}, \quad w = 0.6
$$

---

## 7. Rare Series Handling

A series is considered rare if fraction of nonzero $y_t$:

$$
\frac{\#\{y_t>0\}}{T} < f_{\text{rare}}
$$

- Floor applied:
$$
y_{\text{floor}} = \text{median of last nonzero k values} \cdot f_{\text{floor}}
$$
- Cap growth:
$$
y_{\text{corr}} \le y_T \cdot g_{\text{cap}}
$$

---

## 8. Evaluation: Micro sMAPE

Global metric:
$$
\text{sMAPE} = 100 \cdot \frac{1}{N} \sum_{i=1}^{N} \frac{2 |y_i - \hat y_i|}{|y_i| + |\hat y_i| + \varepsilon}
$$

Top-20 metric: only top 20 countries per `(product × flow)` by **sum of trade_value**.

---

## 9. Summary of Model Flow

1. **Split train/test**  
2. **Identify series** `(product × country × flow)`  
3. **Compute naive candidates** (last, MA, drift, seasonal, same-month median)  
4. **Compute base forecast** $b$ = median(candidates)  
5. **Compute seasonal-trend factors** $S_m$ and slope $\beta$  
6. **Select exogenous features** by Spearman correlation  
7. **Construct features**: lags, seasonality, encodings, exog  
8. **Backtest and fit Huber regression** on $\log$ correction  
9. **Predict next month**: apply Huber correction in log-space  
10. **Rare series adjustment**: floor & growth cap  
11. **Blend with base forecast** for final prediction  
12. **Evaluate with sMAPE** (global and top-20)

---

This approach ensures **robustness, interpretability, and high accuracy**, combining **time series heuristics** with **machine learning residual correction**.




# code d'entrainement  des modèles et clalcul de SMAPE

In [ ]:
import numpy as np
import pandas as pd
from dataclasses import dataclass
from sklearn.linear_model import HuberRegressor
from scipy.stats import spearmanr
import warnings; warnings.filterwarnings("ignore")

# ==================== CONFIG ====================

@dataclass
class Config:
    prod_col: str = "product_id_hs4"
    cntry_col: str = "country_id"
    flow_col: str = "trade_flow_name"
    time_col: str = "month_id"
    target: str   = "trade_value"

    train_start: int = 202101
    train_end:   int = 202102
    test_month:  int = 202103

    # series rules
    min_context: int = 13
    short_series_max: int = 6
    eps_base: float = 1.0

    # scarse data
    rare_nonzero_frac: float = 0.30
    growth_cap_rare: float = 3.0
    floor_frac_rare: float = 0.2

    # local backtest to train huber
    backtest_k: int = 3
    random_state: int = 123

    # Export
    out_xlsx: str = "USA_forcast.xlsx"

    # EXOGen selection
    exog_lags: tuple = (1, 3, 6)
    top_exog_k: int = 8
    exog_cols: list = None

# ==================== UTILS (inchangés) ====================
def smape_micro(yt, yp, eps=1e-8):
    yt, yp = np.asarray(yt, float), np.asarray(yp, float)
    den = np.abs(yt) + np.abs(yp); m = den > 0
    return float(100*np.mean(2*np.abs(yp-yt)[m]/(den[m]+eps))) if m.any() else np.nan

def slope_ols(arr: np.ndarray):
    n = arr.size
    if n == 0 or np.any(~np.isfinite(arr)): return 0.0
    x = np.arange(n, dtype=float)
    x_mean, y_mean = (n - 1) / 2.0, np.nanmean(arr)
    num = np.dot(x - x_mean, arr - y_mean)
    den = np.dot(x - x_mean, x - x_mean)
    return float(num/den) if den > 0 else 0.0

def month_of(yyyymm:int)->int:
    return int(str(int(yyyymm))[-2:])

def is_rare(yhist: pd.Series, min_len=12, frac_thresh=0.30):
    yv = pd.to_numeric(yhist, errors="coerce").fillna(0.0).values
    if len(yv) < min_len:
        return (yv > 0).mean() < frac_thresh
    return (yv > 0).mean() < frac_thresh

def robust_nonzero_stat(yhist: pd.Series, kind="median", k=5):
    yv = pd.to_numeric(yhist, errors="coerce")
    nz = yv[yv > 0]
    if nz.empty: return 0.0
    lastk = nz.tail(min(k, len(nz)))
    return float(lastk.median() if kind=="median" else lastk.mean())

def seasonal_factors_by_month(y: pd.Series, months: pd.Series):
    y = pd.to_numeric(y, errors="coerce")
    nz = y[y > 0]
    gmu = float(nz.median()) if not nz.empty else float(y.replace(0, np.nan).median())
    if not np.isfinite(gmu) or gmu <= 0:
        gmu = float((y.replace(0, np.nan)).mean() or 1.0)
    fac = {}
    for m in range(1,13):
        ym = y[(months==m) & (y>0)]
        medm = float(ym.median()) if not ym.empty else np.nan
        fac[m] = float(np.clip((medm/gmu) if np.isfinite(medm) and gmu>0 else 1.0, 0.2, 5.0))
    return fac, gmu

def same_month_last_k(y: pd.Series, months: pd.Series, target_month:int, k:int=3):
    mask = (months == target_month)
    vals = pd.to_numeric(y[mask], errors="coerce").dropna()
    if vals.empty: return np.nan
    return float(vals.tail(k).median())

def seasonal_trend_forecast(y: pd.Series, months: pd.Series, target_month:int):
    y = pd.to_numeric(y, errors="coerce").fillna(0.0)
    if len(y) == 0: return 0.0
    S, _ = seasonal_factors_by_month(y, months)
    S_series = months.map(S).astype(float).replace(0, 1.0)
    de = y / S_series
    w = 24 if len(de) >= 24 else (12 if len(de) >= 12 else (6 if len(de) >= 6 else len(de)))
    tail = de.tail(w).replace(0, np.nan).dropna()
    if tail.empty:
        level = float(de.iloc[-1]) if len(de)>0 else 0.0
        slope = 0.0
    else:
        level = float(de.iloc[-1])
        slope = slope_ols(np.log1p(tail.values + 1.0))
    de_next = level * np.exp(slope)
    y_next = de_next * S.get(target_month, 1.0)
    samem = same_month_last_k(y, months, target_month, k=3)
    if np.isfinite(samem):
        y_next = np.median([y_next, samem, float(y.iloc[-1])])
    return float(max(0.0, y_next))

def naive_candidates(yhist: pd.Series, months: pd.Series, target_month:int):
    y = pd.to_numeric(yhist, errors="coerce")
    cands = {}
    if len(y) >= 1 and pd.notna(y.iloc[-1]):   cands["last"] = float(y.iloc[-1])
    if len(y) >= 12 and pd.notna(y.iloc[-12]): cands["seas12"] = float(y.iloc[-12])
    if len(y) >= 3:  cands["ma3"] = float(y.iloc[-3:].mean())
    if len(y) >= 6:  cands["ma6"] = float(y.iloc[-6:].mean())
    if len(y) >= 6 and "last" in cands:
        drift = slope_ols(np.log1p(y.iloc[-6:].clip(lower=0).values + 1.0))
        cands["drift_last"] = float(cands["last"] * np.exp(drift))
    cands["seasonal_ST"] = seasonal_trend_forecast(y, months, target_month)
    sm = same_month_last_k(y, months, target_month, k=3)
    if np.isfinite(sm): cands["same_month_med3"] = float(sm)
    return cands

def blend_base_from_candidates(cands: dict):
    vals = [v for v in cands.values() if np.isfinite(v)]
    return float(np.median(vals)) if vals else 0.0

def fit_huber_fast(X, y):
    if len(y) < 20:
        return None
    model = HuberRegressor(alpha=1e-3, epsilon=1.35, max_iter=200)
    model.fit(np.asarray(X), np.asarray(y))
    return model

# ----------------- EXOG -----------------
def detect_exog_cols(df: pd.DataFrame, cfg: Config):
    forbidden = {cfg.time_col, cfg.flow_col, cfg.cntry_col, cfg.prod_col, cfg.target}
    cand = [c for c in df.columns if c not in forbidden]
    num_cand = []
    for c in cand:
        try:
            s = pd.to_numeric(df[c].dropna().iloc[:1000], errors="coerce")
            if s.dropna().size > 0:
                num_cand.append(c)
        except Exception:
            continue
    return num_cand

def select_top_exog_lag_pairs(train_df: pd.DataFrame, cfg: Config, series_col="series_id"):
    exog_cands = cfg.exog_cols or detect_exog_cols(train_df, cfg)
    lags = list(cfg.exog_lags)
    scores = []
    # group by series key in a similar way to the rest of the code (product__country__flow)
    train_df = train_df.copy()
    train_df["__series"] = train_df[cfg.prod_col].astype(str) + "__" + train_df[cfg.cntry_col].astype(str) + "__" + train_df[cfg.flow_col].astype(str)
    grouped = train_df.sort_values(["__series", cfg.time_col]).groupby("__series")
    for col in exog_cands:
        for L in lags:
            xs, ys = [], []
            for _, g in grouped:
                if col not in g.columns:
                    continue
                gcol = pd.to_numeric(g[col], errors="coerce")
                gtarget = pd.to_numeric(g[cfg.target], errors="coerce")
                if len(g) <= L:
                    continue
                ex = gcol.shift(L).iloc[L:].values
                tar = gtarget.iloc[L:].values
                mask = np.isfinite(ex) & np.isfinite(tar)
                if mask.any():
                    xs.append(ex[mask])
                    ys.append(tar[mask])
            if not xs:
                continue
            x_all = np.concatenate(xs)
            y_all = np.concatenate(ys)
            if len(x_all) < 10:
                continue
            try:
                corr, _ = spearmanr(x_all, y_all, nan_policy="omit")
                if np.isfinite(corr):
                    scores.append((abs(float(corr)), col, L))
            except Exception:
                continue
    scores_sorted = sorted(scores, key=lambda t: t[0], reverse=True)
    topk = scores_sorted[: cfg.top_exog_k]
    selected = [(col, lag) for _, col, lag in topk]
    return selected

# Global selected exog list (populated in forecast function)
SELECTED_EXOG_PAIRS = []

# ----------------- PASSE-PARTOUT: ajout safe d'exogènes dans feats ------------
def add_exog_feats_to_feats_dict(feats: dict, hist_df: pd.DataFrame):
    # uses SELECTED_EXOG_PAIRS global
    for col, lag in SELECTED_EXOG_PAIRS:
        key = f"exog_lag_{col}_L{lag}"
        try:
            if col in hist_df.columns:
                ser = pd.to_numeric(hist_df[col], errors="coerce")
                # prefer exact -lag value if available, otherwise last non-na
                if len(ser) >= lag and pd.notna(ser.iloc[-lag]):
                    val = float(ser.iloc[-lag])
                else:
                    snotna = ser.dropna()
                    val = float(snotna.iloc[-1]) if not snotna.empty else 0.0
            else:
                val = 0.0
        except Exception:
            val = 0.0
        feats[key] = float(val)
    return feats

# ==================== TRAINING + PERFORMANCE ====================
def forecast_train(data: pd.DataFrame, cfg: Config = Config()):
    global mu, enc_prod, enc_cntry, enc_flow, SELECTED_EXOG_PAIRS

    PROD, CNTRY, FLOW, TIME, TARGET = cfg.prod_col, cfg.cntry_col, cfg.flow_col, cfg.time_col, cfg.target
    SERIES = "series_id"
    np.random.seed(cfg.random_state)

    df = data.copy()

    need = {PROD, CNTRY, FLOW, TIME, TARGET}
    assert need.issubset(df.columns), f"Colonnes manquantes: {list(need - set(df.columns))}"

    # types & month_num
    for c in (PROD, CNTRY, FLOW): df[c] = df[c].astype(str)
    df[TIME] = pd.to_numeric(df[TIME], errors="coerce").astype(int)
    df["month_num"] = df[TIME].map(month_of)

    # split
    train = df[(df[TIME] >= cfg.train_start) & (df[TIME] <= cfg.train_end)].copy()
    test  = df[df[TIME] == cfg.test_month].copy()
    assert not train.empty and not test.empty, "Vérifie la présence des mois exigés."

    # series id
    train[SERIES] = train[PROD] + "__" + train[CNTRY] + "__" + train[FLOW]
    test[SERIES]  = test[PROD]  + "__" + test[CNTRY]  + "__" + test[FLOW]
    ids = sorted(set(train[SERIES]) & set(test[SERIES]))
    train = train[train[SERIES].isin(ids)].sort_values([SERIES, TIME]).reset_index(drop=True)
    test  = test[test[SERIES].isin(ids)].sort_values([SERIES, TIME]).reset_index(drop=True)

    # encod
    mu = float(train[TARGET].mean())
    enc_prod  = train.groupby(PROD)[TARGET].mean().to_dict()
    enc_cntry = train.groupby(CNTRY)[TARGET].mean().to_dict()
    enc_flow  = train.groupby(FLOW)[TARGET].mean().to_dict()

    # detect exog candidates if not provided
    if cfg.exog_cols is None:
        cfg.exog_cols = detect_exog_cols(df, cfg)

    # select top exog (col,lag) using only train
    SELECTED_EXOG_PAIRS = select_top_exog_lag_pairs(train, cfg, series_col=SERIES)
    print("Selected exog (col,lag):", SELECTED_EXOG_PAIRS)

    # Build per_series_hist but now include exog columns so feats function can read them
    extra_cols = [c for (c,_) in SELECTED_EXOG_PAIRS if c in train.columns]
    per_series_hist = {
        sid: g[[TIME, TARGET, PROD, CNTRY, FLOW, "month_num"] + extra_cols].sort_values(TIME).reset_index(drop=True)
        for sid, g in train.groupby(SERIES)
    }

    # Backtest short for Huber
    rows_X, rows_y, feature_order = [], [], None

    for sid, hist in per_series_hist.items():
        y = pd.to_numeric(hist[TARGET], errors="coerce")
        if len(y) <= cfg.short_series_max:
            continue
        K = min(cfg.backtest_k, len(y))
        for k in range(1, K+1):
            yhist = y.iloc[: -k]
            mser  = hist["month_num"].iloc[: -k]
            base  = blend_base_from_candidates(naive_candidates(yhist, mser, target_month=int(hist.iloc[-k]["month_num"])))
            if base <= 0 and len(yhist)>0:
                base = max(1e-8, float(yhist.iloc[-1]))
            prod, cntry, flow = hist.iloc[-k][PROD], hist.iloc[-k][CNTRY], hist.iloc[-k][FLOW]
            month = int(hist.iloc[-k][TIME]) if np.isscalar(hist.iloc[-k][TIME]) else int(hist.iloc[-k][TIME].item())

            # original features
            y_hist_df = hist.iloc[: -k]
            S, _ = seasonal_factors_by_month(y_hist_df[TARGET], y_hist_df["month_num"])
            feats = {
                "month": int(str(month)[-2:]),
                "is_jan": int(str(month).endswith("01")),
                "is_dec": int(str(month).endswith("12")),
                "enc_prod":  enc_prod.get(prod, mu),
                "enc_cntry": enc_cntry.get(cntry, mu),
                "enc_flow":  enc_flow.get(flow, mu),
                "season_factor": float(S.get(int(str(month)[-2:]), 1.0)),
            }
            for L in (1,2,3,6,12):
                feats[f"lag_{L}"] = float(y.iloc[-L]) if len(y)>=L and pd.notna(y.iloc[-L]) else 0.0

            # --- AJOUT: exogènes sélectionnés (valeur last available respecting lag) ---
            feats = add_exog_feats_to_feats_dict(feats, y_hist_df)

            if feature_order is None:
                feature_order = list(feats.keys())
            rows_X.append([feats[k] for k in feature_order])
            rows_y.append(np.log1p(float(y.iloc[-k]) + cfg.eps_base) - np.log1p(base + cfg.eps_base))

    huber = fit_huber_fast(rows_X, rows_y)

    # Prediction for test_month
    pred_rows = []
    last_obs_value_map, last_obs_month_map = {}, {}

    # rebuild per_series_hist for full train (now with exog cols)
    per_series_hist = {
        sid: g[[TIME, TARGET, PROD, CNTRY, FLOW, "month_num"] + extra_cols].sort_values(TIME).reset_index(drop=True)
        for sid, g in train.groupby(SERIES)
    }

    for sid, hist in per_series_hist.items():
        # last obs safe
        last_obs_value = float(pd.to_numeric(hist[TARGET], errors="coerce").dropna().iloc[-1]) if not hist.empty else np.nan
        last_obs_month = int(hist[TIME].dropna().iloc[-1]) if not hist.empty else np.nan
        last_obs_value_map[sid] = last_obs_value
        last_obs_month_map[sid] = last_obs_month

        yhist = pd.to_numeric(hist[TARGET], errors="coerce")
        mser  = hist["month_num"]

        if len(hist) <= cfg.short_series_max:
            base = blend_base_from_candidates(naive_candidates(yhist, mser, target_month=month_of(cfg.test_month)))
            y_pred = max(0.0, float(base))
        else:
            cands = naive_candidates(yhist, mser, target_month=month_of(cfg.test_month))
            base  = blend_base_from_candidates(cands)
            if base <= 0 and len(yhist)>0:
                base = max(1e-8, float(yhist.iloc[-1]))

            prod, cntry, flow = hist.iloc[-1][PROD], hist.iloc[-1][CNTRY], hist.iloc[-1][FLOW]

            # build feats as in backtest but on full hist
            S, _ = seasonal_factors_by_month(pd.to_numeric(hist[TARGET], errors="coerce"), hist["month_num"])
            feats = {
                "month": month_of(cfg.test_month),
                "is_jan": int(str(cfg.test_month).endswith("01")),
                "is_dec": int(str(cfg.test_month).endswith("12")),
                "enc_prod":  enc_prod.get(prod, mu),
                "enc_cntry": enc_cntry.get(cntry, mu),
                "enc_flow":  enc_flow.get(flow, mu),
                "season_factor": float(S.get(month_of(cfg.test_month), 1.0)),
            }
            for L in (1,2,3,6,12):
                feats[f"lag_{L}"] = float(yhist.iloc[-L]) if len(yhist)>=L and pd.notna(yhist.iloc[-L]) else 0.0

            feats = add_exog_feats_to_feats_dict(feats, hist)

            Xrow = np.array([[feats[k] for k in feature_order]]) if feature_order else None
            delta = float(huber.predict(Xrow)[0]) if (huber is not None and Xrow is not None) else 0.0
            y_corr = float(np.expm1(np.log1p(base + cfg.eps_base) + delta))

            # rare series handling
            if is_rare(yhist, frac_thresh=cfg.rare_nonzero_frac):
                est = robust_nonzero_stat(yhist, "median", k=5)
                floor = max(1e-8, cfg.floor_frac_rare * est)
                y_corr = max(y_corr, floor)
                if len(yhist) >= 1:
                    prev = float(yhist.iloc[-1])
                    y_corr = min(y_corr, prev * cfg.growth_cap_rare)

            w = 0.6
            y_pred = max(0.0, w*y_corr + (1.0 - w)*base)

        # de-duplicate test rows for safety (avoid duplicates causing reindex errors)
        te_rows = test[test[SERIES]==sid].drop_duplicates(subset=[PROD, CNTRY, FLOW, TIME])
        for _, r in te_rows.iterrows():
            time_val = int(r[TIME]) if np.isscalar(r[TIME]) else int(r[TIME].item())
            pred_rows.append((
                time_val, str(r[PROD]), str(r[CNTRY]), str(r[FLOW]),
                float(r.get(TARGET, np.nan)), float(y_pred),
                float(last_obs_value_map[sid]), int(last_obs_month_map[sid])
            ))

    pred_df = pd.DataFrame(
        pred_rows,
        columns=[cfg.time_col, cfg.prod_col, cfg.cntry_col, cfg.flow_col, "y_true", "y_pred", "last_obs_value", "last_obs_month"]
    ).sort_values([cfg.time_col, cfg.prod_col, cfg.cntry_col, cfg.flow_col]).reset_index(drop=True)

    # ==== sMAPE GLOBAL ====
    if pred_df["y_true"].notna().any():
        smape_all = smape_micro(pred_df["y_true"], pred_df["y_pred"])
        print(f"sMAPE (micro) GLOBAL test {cfg.test_month}: {smape_all:.3f}%  | lignes: {pred_df['y_true'].notna().sum()}/{len(pred_df)}")
    else:
        smape_all = np.nan

    # TOP-20 ===
    test_true = df[df[cfg.time_col] == cfg.test_month][[cfg.prod_col, cfg.cntry_col, cfg.flow_col, cfg.time_col, cfg.target]].copy()

    agg = (test_true
           .groupby([cfg.prod_col, cfg.flow_col, cfg.time_col, cfg.cntry_col], as_index=False)[cfg.target]
           .sum()
           .rename(columns={cfg.target: "sum_trade_value"}))

    agg = agg.sort_values([cfg.prod_col, cfg.flow_col, cfg.time_col, "sum_trade_value"],
                          ascending=[True, True, True, False])
    top20 = agg.groupby([cfg.prod_col, cfg.flow_col, cfg.time_col]).head(20)

    pred_top = pred_df.merge(
        top20[[cfg.prod_col, cfg.cntry_col, cfg.flow_col, cfg.time_col]],
        on=[cfg.prod_col, cfg.cntry_col, cfg.flow_col, cfg.time_col],
        how="inner"
    ).reset_index(drop=True)

    # ==== sMAPE TOP-20 ====
    if pred_top["y_true"].notna().any():
        smape_top20 = smape_micro(pred_top["y_true"], pred_top["y_pred"])
        print(f"sMAPE (micro) TOP-20 par produit×flow×mois {cfg.test_month}: {smape_top20:.3f}%  | lignes: {pred_top['y_true'].notna().sum()}/{len(pred_top)}")
    else:
        smape_top20 = np.nan

    # ==== Export Excel ====
    with pd.ExcelWriter(cfg.out_xlsx, engine="xlsxwriter") as wr:
        pred_top.to_excel(wr, sheet_name="predictions_top20", index=False)
    print(f"📁 Fichier Excel sauvegardé (TOP-20 par produit×flow×mois) → {cfg.out_xlsx}")

    return pred_df, pred_top, smape_all, smape_top20




# ==================== test ====================
def forecast_test(data: pd.DataFrame, cfg: Config = Config()):
    global mu, enc_prod, enc_cntry, enc_flow, SELECTED_EXOG_PAIRS

    PROD, CNTRY, FLOW, TIME, TARGET = cfg.prod_col, cfg.cntry_col, cfg.flow_col, cfg.time_col, cfg.target
    SERIES = "series_id"
    np.random.seed(cfg.random_state)

    df = data.copy()

    need = {PROD, CNTRY, FLOW, TIME, TARGET}
    assert need.issubset(df.columns), f"Colonnes manquantes: {list(need - set(df.columns))}"

    # types & month_num
    for c in (PROD, CNTRY, FLOW): df[c] = df[c].astype(str)
    df[TIME] = pd.to_numeric(df[TIME], errors="coerce").astype(int)
    df["month_num"] = df[TIME].map(month_of)

    # split
    train = df[(df[TIME] >= cfg.train_start) & (df[TIME] <= cfg.train_end)].copy()
    test  = df[df[TIME] == cfg.test_month].copy()
    assert not train.empty and not test.empty, "Vérifie la présence des mois exigés."

    # series id
    train[SERIES] = train[PROD] + "__" + train[CNTRY] + "__" + train[FLOW]
    test[SERIES]  = test[PROD]  + "__" + test[CNTRY]  + "__" + test[FLOW]
    ids = sorted(set(train[SERIES]) & set(test[SERIES]))
    train = train[train[SERIES].isin(ids)].sort_values([SERIES, TIME]).reset_index(drop=True)
    test  = test[test[SERIES].isin(ids)].sort_values([SERIES, TIME]).reset_index(drop=True)

    # mean encod
    mu = float(train[TARGET].mean())
    enc_prod  = train.groupby(PROD)[TARGET].mean().to_dict()
    enc_cntry = train.groupby(CNTRY)[TARGET].mean().to_dict()
    enc_flow  = train.groupby(FLOW)[TARGET].mean().to_dict()

    # detect exog candidates if not provided
    if cfg.exog_cols is None:
        cfg.exog_cols = detect_exog_cols(df, cfg)

    # select top exog (col,lag) using only train
    SELECTED_EXOG_PAIRS = select_top_exog_lag_pairs(train, cfg, series_col=SERIES)
    print("Selected exog (col,lag):", SELECTED_EXOG_PAIRS)

    # Build per_series_hist but now include exog columns so feats function can read them
    extra_cols = [c for (c,_) in SELECTED_EXOG_PAIRS if c in train.columns]
    per_series_hist = {
        sid: g[[TIME, TARGET, PROD, CNTRY, FLOW, "month_num"] + extra_cols].sort_values(TIME).reset_index(drop=True)
        for sid, g in train.groupby(SERIES)
    }

    # Backtest short for Huber
    rows_X, rows_y, feature_order = [], [], None

    for sid, hist in per_series_hist.items():
        y = pd.to_numeric(hist[TARGET], errors="coerce")
        if len(y) <= cfg.short_series_max:
            continue
        K = min(cfg.backtest_k, len(y))
        for k in range(1, K+1):
            yhist = y.iloc[: -k]
            mser  = hist["month_num"].iloc[: -k]
            base  = blend_base_from_candidates(naive_candidates(yhist, mser, target_month=int(hist.iloc[-k]["month_num"])))
            if base <= 0 and len(yhist)>0:
                base = max(1e-8, float(yhist.iloc[-1]))
            prod, cntry, flow = hist.iloc[-k][PROD], hist.iloc[-k][CNTRY], hist.iloc[-k][FLOW]
            month = int(hist.iloc[-k][TIME]) if np.isscalar(hist.iloc[-k][TIME]) else int(hist.iloc[-k][TIME].item())

            # original features
            y_hist_df = hist.iloc[: -k]
            S, _ = seasonal_factors_by_month(y_hist_df[TARGET], y_hist_df["month_num"])
            feats = {
                "month": int(str(month)[-2:]),
                "is_jan": int(str(month).endswith("01")),
                "is_dec": int(str(month).endswith("12")),
                "enc_prod":  enc_prod.get(prod, mu),
                "enc_cntry": enc_cntry.get(cntry, mu),
                "enc_flow":  enc_flow.get(flow, mu),
                "season_factor": float(S.get(int(str(month)[-2:]), 1.0)),
            }
            for L in (1,2,3,6,12):
                feats[f"lag_{L}"] = float(y.iloc[-L]) if len(y)>=L and pd.notna(y.iloc[-L]) else 0.0

            # --- add exog ---
            feats = add_exog_feats_to_feats_dict(feats, y_hist_df)

            if feature_order is None:
                feature_order = list(feats.keys())
            rows_X.append([feats[k] for k in feature_order])
            rows_y.append(np.log1p(float(y.iloc[-k]) + cfg.eps_base) - np.log1p(base + cfg.eps_base))

    huber = fit_huber_fast(rows_X, rows_y)

    # Prediction for test_month
    pred_rows = []
    last_obs_value_map, last_obs_month_map = {}, {}

    # rebuild per_series_hist for full train (now with exog cols)
    per_series_hist = {
        sid: g[[TIME, TARGET, PROD, CNTRY, FLOW, "month_num"] + extra_cols].sort_values(TIME).reset_index(drop=True)
        for sid, g in train.groupby(SERIES)
    }

    for sid, hist in per_series_hist.items():
        # last obs safe
        last_obs_value = float(pd.to_numeric(hist[TARGET], errors="coerce").dropna().iloc[-1]) if not hist.empty else np.nan
        last_obs_month = int(hist[TIME].dropna().iloc[-1]) if not hist.empty else np.nan
        last_obs_value_map[sid] = last_obs_value
        last_obs_month_map[sid] = last_obs_month

        yhist = pd.to_numeric(hist[TARGET], errors="coerce")
        mser  = hist["month_num"]

        if len(hist) <= cfg.short_series_max:
            base = blend_base_from_candidates(naive_candidates(yhist, mser, target_month=month_of(cfg.test_month)))
            y_pred = max(0.0, float(base))
        else:
            cands = naive_candidates(yhist, mser, target_month=month_of(cfg.test_month))
            base  = blend_base_from_candidates(cands)
            if base <= 0 and len(yhist)>0:
                base = max(1e-8, float(yhist.iloc[-1]))

            prod, cntry, flow = hist.iloc[-1][PROD], hist.iloc[-1][CNTRY], hist.iloc[-1][FLOW]

            # build feats as in backtest but on full hist
            S, _ = seasonal_factors_by_month(pd.to_numeric(hist[TARGET], errors="coerce"), hist["month_num"])
            feats = {
                "month": month_of(cfg.test_month),
                "is_jan": int(str(cfg.test_month).endswith("01")),
                "is_dec": int(str(cfg.test_month).endswith("12")),
                "enc_prod":  enc_prod.get(prod, mu),
                "enc_cntry": enc_cntry.get(cntry, mu),
                "enc_flow":  enc_flow.get(flow, mu),
                "season_factor": float(S.get(month_of(cfg.test_month), 1.0)),
            }
            for L in (1,2,3,6,12):
                feats[f"lag_{L}"] = float(yhist.iloc[-L]) if len(yhist)>=L and pd.notna(yhist.iloc[-L]) else 0.0

            feats = add_exog_feats_to_feats_dict(feats, hist)

            Xrow = np.array([[feats[k] for k in feature_order]]) if feature_order else None
            delta = float(huber.predict(Xrow)[0]) if (huber is not None and Xrow is not None) else 0.0
            y_corr = float(np.expm1(np.log1p(base + cfg.eps_base) + delta))

            # rare series handling
            if is_rare(yhist, frac_thresh=cfg.rare_nonzero_frac):
                est = robust_nonzero_stat(yhist, "median", k=5)
                floor = max(1e-8, cfg.floor_frac_rare * est)
                y_corr = max(y_corr, floor)
                if len(yhist) >= 1:
                    prev = float(yhist.iloc[-1])
                    y_corr = min(y_corr, prev * cfg.growth_cap_rare)

            w = 0.6
            y_pred = max(0.0, w*y_corr + (1.0 - w)*base)

        # de-duplicate test rows for safety (avoid duplicates causing reindex errors)
        te_rows = test[test[SERIES]==sid].drop_duplicates(subset=[PROD, CNTRY, FLOW, TIME])
        for _, r in te_rows.iterrows():
            time_val = int(r[TIME]) if np.isscalar(r[TIME]) else int(r[TIME].item())
            pred_rows.append((
                time_val, str(r[PROD]), str(r[CNTRY]), str(r[FLOW]),
                 float(y_pred),
                float(last_obs_value_map[sid]), int(last_obs_month_map[sid])
            ))

    pred_df = pd.DataFrame(
        pred_rows,
        columns=[cfg.time_col, cfg.prod_col, cfg.cntry_col, cfg.flow_col,  "y_pred", "last_obs_value", "last_obs_month"]
    ).sort_values([cfg.time_col, cfg.prod_col, cfg.cntry_col, cfg.flow_col]).reset_index(drop=True)


       # ==== Export Excel ====
    with pd.ExcelWriter(cfg.out_xlsx, engine="xlsxwriter") as wr:
        pred_df.to_excel(wr, sheet_name="predictions_202510", index=False)
    print(f"pred_tr → {cfg.out_xlsx}")

    return pred_df


USA

In [ ]:
@dataclass
class Config_usa:
    prod_col: str = "product_id_hs4"
    cntry_col: str = "country_id"
    flow_col: str = "trade_flow_name"
    time_col: str = "month_id"
    target: str   = "trade_value"

    train_start: int = 202101
    train_end:   int = 202102
    test_month:  int = 202103

    # series rules
    min_context: int = 13
    short_series_max: int = 6
    eps_base: float = 1.0

    # scarse data
    rare_nonzero_frac: float = 0.30
    growth_cap_rare: float = 3.0
    floor_frac_rare: float = 0.2

    # local backtest to train huber
    backtest_k: int = 3
    random_state: int = 123

    # Export
    out_xlsx: str = "USA_forcast.xlsx"

    # EXOGen selection
    exog_lags: tuple = (1, 3, 6)
    top_exog_k: int = 8
    exog_cols: list = None


#pred_USA= forecast_test(USA
pred_USA=forecast_test(data=USA, cfg= Config_usa())

#china

In [ ]:
@dataclass
class Config_china:
    prod_col: str = "product_id_hs4"
    cntry_col: str = "country_id"
    flow_col: str = "trade_flow_name"
    time_col: str = "month_id"
    target: str   = "trade_value"

    train_start: int = 202101
    train_end:   int = 202509
    test_month:  int = 202510

    # Règles séries
    min_context: int = 13
    short_series_max: int = 6
    eps_base: float = 1.0

    # Séries rares
    rare_nonzero_frac: float = 0.30
    growth_cap_rare: float = 3.0
    floor_frac_rare: float = 0.2

    # Petit backtest local pour entraîner Huber
    backtest_k: int = 3
    random_state: int = 123

    # Export
    out_xlsx: str = "china_forcast.xlsx"

    # EXOGènes selection
    exog_lags: tuple = (1, 3, 6)
    top_exog_k: int = 8
    exog_cols: list = None   # si None => auto-detect depuis USA

pred_USA=forecast_test(data=china, cfg= Config_china())
